In [3]:
# Config
from dotenv import load_dotenv
import os 
import sys
import google.protobuf.text_format as tf
import re
from bareunpy import Tagger
from bareunpy import Tokenizer
import nltk
from tqdm import tqdm
tqdm.pandas() 


nltk.download('stopwords')

load_dotenv('../../')   # load .env
API_KEY = os.environ.get('BAREUN_API_KEY') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/haseong8012/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import os

print(os.getcwd())

/mnt/hdd/haseong8012/thread/src/preprocessing


In [5]:
# Load data
import pandas as pd

data_path = "../../data/political_news/political_news_20250409_2109.csv" 
df = pd.read_csv(data_path)
df

,title,description,published,link,media
0,"韓대행 “美 관세, 보복이 아니라 소통과 한미 동맹 격상이 해법”","<img alt=""한덕수 대통령 권한대행 국무총리가 지난 8일 정부서울청사 집무실에...","Wed, 09 Apr 2025 11:14:25 +0000",https://www.chosun.com/politics/goverment/2025...,조선닷컴
1,선관위원장 “대선 때 수검표... 사전투표함 보관소 24시간 영상 공개”,"<img alt=""노태악 중앙선거관리위원장이 9일 오전 경기 과천시 중앙선거관리위원...","Wed, 09 Apr 2025 08:00:22 +0000",https://www.chosun.com/politics/politics_gener...,조선닷컴
2,"이완규, ‘국민의힘 당적 논란’에 “정당 가입한 적 없다”",<p>헌법재판관 후보자로 지명된 이완규 법제처장은 민주당 일각에서 제기된 ‘국민의힘...,"Wed, 09 Apr 2025 03:42:04 +0000",https://www.chosun.com/national/court_law/2025...,조선닷컴
3,"민주당 초선들 “韓, 이완규·함상훈 지명 철회 안하면 탄핵 추진""","<img alt=""더불어민주당 초선의원 모임인 '더민초' 소속 의원들이 9일 오후 ...","Wed, 09 Apr 2025 07:15:17 +0000",https://www.chosun.com/politics/assembly/2025/...,조선닷컴
4,"‘권한대행 헌법재판관 지명 금지’, 민주당 주도 법사위 통과",<p>대통령 권한대행이 대통령 몫의 헌법재판관을 지명할 수 없도록 하는 헌법재판소법...,"Wed, 09 Apr 2025 02:03:47 +0000",https://www.chosun.com/politics/assembly/2025/...,조선닷컴
...,...,...,...,...,...
458,"한 대행 '계엄 피의자' 재판관 지명에…민주 ""원천 무효"" 반발",[앵커]\n\n한덕수 대통령 권한대행이 대통령 몫인 헌법재판관 후보자 두 명을 기습...,"Wed, 09 Apr 2025 08:02:51 +0900",https://news.jtbc.co.kr/article/NB12241976,JTBC
459,오늘의 주요뉴스,1. 한-트럼프 첫 통화&hellip;방위비&middot;관세 논의\n\n한덕수 대...,"Wed, 09 Apr 2025 07:38:41 +0900",https://news.jtbc.co.kr/article/NB12241986,JTBC
460,한덕수-트럼프 28분 첫 통화…관세·방위비 등 압박,8일 밤 오후 9시 3분.\n한덕수 대통령 권한대행 국무총리와 도널드 트럼프 미국 ...,"Wed, 09 Apr 2025 00:41:34 +0900",https://news.jtbc.co.kr/article/NB12241970,JTBC
461,"한 대행 CNN 인터뷰…""미 관세, 맞서지 않고 협상할 것""",미국 CNN과 인터뷰한 한덕수 대통령 권한대행 국무총리. 〈사진=CNN 캡처〉한덕수...,"Wed, 09 Apr 2025 00:00:29 +0900",https://news.jtbc.co.kr/article/NB12241969,JTBC


In [14]:
import numpy as np 
from tqdm.notebook import tqdm
tqdm.pandas()

def tokenize(tokenizer, text):
    """
    Tokenize the text using the Bareun tokenizer.
    """
    if isinstance(text, str) and text.strip():  # 문자열인지 확인하고 비어 있지 않은지 확인
        result = tokenizer.tokenize(text).segments()
        return result
    return np.nan  # text가 유효하지 않으면 np.nan 반환


# toekenize
tokenizer = Tokenizer(API_KEY, 'localhost', port=5757)
tokenized = df["description"].progress_apply(lambda x: tokenize(tokenizer, x))
print(tokenized[0])

  0%|          | 0/463 [00:00<?, ?it/s]

['<', 'img', 'alt', '=', '"', '한덕수', '대통령', '권한', '대행', '국무총리', '가', '지나', 'ㄴ', '8', '일', '정부서울청사', '집무실', '에서', '도널드', '트럼프', '미국', '대통령', '과', '통화', '하', '고', '있', '다', '(', '왼쪽', '사진', ')', '.', '지나', 'ㄴ', '1', '월', '트럼프', '대통령', '취임', '이후', '한미', '정상', '간', '직', '접', '소통', '은', '이번', '이', '처음', '이', '다', '.', '통화', '는', '28', '분', '간', '이뤄지', '었', '다고', '국무총리실', '은', '밝히', '었', '다', '.', '오른쪽', '은', '트럼프', '대통령', '이', '2018', '년', '11', '월', '트럼프', '행정부', '1', '기', '때', '통화', '하', '는', '모습', '.', '/', '국무총리실', '·', 'AP', '연합', '뉴스', '"', 'height', '=', '"', '1000', '"', 'src', '="', 'https', '://', 'www', '.', 'chosun', '.', 'com', '/', 'resizer', '/', 'v', '2', '/', 'Q', '7', 'PR', '7', 'IMD', '3', 'JESNJPS', '7', 'X', '7', 'UXOM', '7', 'WQ', '.', 'jpg', '?', 'width', '=', '1600', '&', 'amp', ';', 'height', '=', '1000', '&', 'amp', ';', 'auth', '=', 'a', '5830', 'd', '623', 'e5', 'ed', '4', 'c', '3557', 'd', '36', 'c', '8', 'd', '8', 'bac', '4913', 'cda', '9c', '87', 'f4', 'd', '5

In [ ]:
# If you have your own localhost bareun.
tokenizer = Tokenizer(API_KEY, 'localhost', port=5757)

# print results.
text ="안녕하세요. 반가워요! 이 문장은 테스트 문장입니다."
tokenized = tokenizer.tokenize(text)
# m = tokenized.msg()
# tmp = tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
# print("Tokenized Text:", tmp)

jo = tokenized.segments()
print(jo)




['안녕', '하', '시', '어요', '.', '반갑', '어요', '!', '이', '문장', '은', '테스트', '문장', '이', 'ㅂ니다', '.']
